# PPA3 Database Table Viewer
If PPA is live and people are using it, and you want to view tables, please do so through this tool rather than Arc Pro. Viewing in Pro will create a schema lock on the tables, so when a user runs the tool, it will cause an error because it cannot log the run output to the locked database tables.

This viewer gets around this issue by creating an in-memory copy of desired gdb tables.

In [7]:
import pandas as pd

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

file_gdb = r'\\arcserver-svr\D\PPA3_SVR\PPA3_GIS_SVR\PPA3_run_data.gdb'
arcpy.env.workspace = file_gdb

fc_pmaster = 'project_master'
f_uid = 'project_uid'

df_pmaster = pd.DataFrame.spatial.from_featureclass(fc_pmaster)

def esri_object_to_df(in_esri_obj, esri_obj_fields, index_field=None):
    '''converts esri gdb table, feature class, feature layer, or SHP to pandas dataframe'''
    data_rows = []
    with arcpy.da.SearchCursor(in_esri_obj, esri_obj_fields) as cur:
        for row in cur:
            out_row = list(row)
            data_rows.append(out_row)

    out_df = pd.DataFrame(data_rows, index=index_field, columns=esri_obj_fields)
    return out_df

print([t for t in arcpy.ListTables()])


['rp_artexp_vmt', 'rp_artexp_econ', 'rp_artexp_eq', 'rp_artexp_mm', 'rp_artexp_sgr', 'rp_fwy_vmt', 'rp_fwy_cong', 'rp_fwy_mm', 'rp_fwy_econ', 'rp_fwy_frgt', 'rp_fwy_saf', 'rp_artsgr_sgr', 'cd_compactdev', 'cd_mixeduse', 'cd_houschoice', 'cd_naturpres', 'rp_artexp_cong', 'rp_artexp_frgt', 'rp_artexp_saf', 'cd_trnchoice', 'cd_existgasset', 'TEST_TABLE']


## View of most recent entries into master table

In [8]:
fields = ['proj_name','juris','proj_type','comm_type','time_created','user_email']
df_pmaster[fields].sort_values(by='time_created', ascending=False).head(10)

,proj_name,juris,proj_type,comm_type,time_created,user_email
269,Maintenance Paving Project,County of Sutter,Arterial State of Good Repair,Ag,2023-01-24 11:27:53.000001,LCarrillo@co.sutter.ca.us
268,8. Watt I-80 Station,SacRT,Community Design,Arterials & Suburban Corridors,2023-01-24 10:44:19.000001,gballard-rosa@sacog.org
267,7. Sac Valley Station,SacRT,Community Design,Urban core,2023-01-24 10:42:45.000001,gballard-rosa@sacog.org
266,6. Folsom Blvd,Sac County,Community Design,Arterials & Suburban Corridors,2023-01-24 10:40:39.000000,gballard-rosa@sacog.org
265,5. Arden Way,Sac County,Community Design,Arterials & Suburban Corridors,2023-01-24 10:38:35.000000,gballard-rosa@sacog.org
264,4. Setzer Tunnel,Sac City,Community Design,Established Communities,2023-01-24 10:37:21.000000,gballard-rosa@sacog.org
263,3. El Camino Ave,Sac City,Community Design,Arterials & Suburban Corridors,2023-01-24 10:36:12.000000,gballard-rosa@sacog.org
262,2. Broadway Phase 3,Sac City,Community Design,Arterials & Suburban Corridors,2023-01-24 10:29:47.000000,gballard-rosa@sacog.org
261,1. Auburn Blvd Phase 2,Citrus Heights,Community Design,Arterials & Suburban Corridors,2023-01-24 10:28:29.000000,gballard-rosa@sacog.org
260,Pennington Road Complete Streets Rehabilitation,City of Live Oak,Arterial State of Good Repair,Small-Town Established Communities,2023-01-24 09:22:14.000000,srolls@rarcivil.com


## Find UID values that are in any of the data tables but not in the master table

In [3]:


def get_uids(in_tbl):
    uids = []

    try:
        with arcpy.da.SearchCursor(in_tbl, field_names=[f_uid]) as cur:
            for row in cur:
                uids.append(row[0])
    except:
        print(f"{in_tbl} does not have field {f_uid}. Skipping...")
        
    return uids

data_tables = [t for t in arcpy.ListTables()]
pmaster_uids = df_pmaster[f_uid].to_list()

out_dict = {}

fake_id = 'UID_NOT_FOUND' # placeholder ID used when running tool locally for testing

for t in data_tables:
    tbl_uids = get_uids(t)
    
    uids_not_in_master = []
    for uid in tbl_uids:
        if uid not in pmaster_uids and uid != fake_id:
            uids_not_in_master.append(uid)
            
    out_dict[t] = uids_not_in_master
    if len(out_dict[t]) > 0:
        print(f"table {t} has the follwing UIDs not in the project_master table: {out_dict[t]}")
        
total_missing = sum([len(v) for tbl, v in out_dict.items()])
if total_missing == 0: print("no IDs found in subreport tables that are not also in project_master")
    

TEST_TABLE does not have field project_uid. Skipping...
no IDs found in subreport tables that are not also in project_master


## Show specific subreport data and for specific projects

In [4]:

subrpt_tbl = 'rp_artexp_cong'
project_uid = None

fields = [f.name for f in arcpy.ListFields(subrpt_tbl)]

#uncomment if you want to filter which fields appear
# fields = [f_uid, 'crash_cnt', 'crash_100mvmt'] # [f.name for f in arcpy.ListFields(subrpt_tbl)]

df_master_fields = [f_uid, 'time_created', 'user_email', 'len_mi', 'aadt']
df_subrpt = esri_object_to_df(subrpt_tbl, esri_obj_fields=fields, index_field=None)


df_jn = df_pmaster[df_master_fields].merge(df_subrpt, on=f_uid, suffixes=('','_y')) \
    .sort_values(by='time_created', ascending=False)
df_jn.head(10)

,project_uid,time_created,user_email,len_mi,aadt,OBJECTID,aadt_y,congspd_nb,congspd_sb,congspd_eb,...,jobs_base,jobs_future,du_base,du_future,congspd_wrst,congrat_wrst,lottr_ampk_wrst,lottr_midday_wrst,lottr_pmpk_wrst,lottr_wknd_wrst
119,b1272ec4-f7c9-4d3c-84d4-b6a00e0e1878,2023-01-24 04:38:49.000000,molly.tremblay@kimley-horn.com,0.727929,21556.0,288,29428,NaN,NaN,24.328146,...,NaN,NaN,NaN,NaN,21.641535,0.561347,1.369187,1.332788,1.481877,1.362365
118,b1272ec4-f7c9-4d3c-84d4-b6a00e0e1878,2023-01-24 04:38:49.000000,molly.tremblay@kimley-horn.com,0.727929,21556.0,287,21556,NaN,NaN,24.328131,...,NaN,NaN,NaN,NaN,21.641323,0.561344,1.369194,1.332792,1.481890,1.362371
117,835fa111-aeb0-4220-a27c-00a6f970a6a7,2023-01-23 15:17:57.000001,maasd@saccounty.gov,12.985438,30000.0,286,30000,25.182341,27.068140,NaN,...,NaN,NaN,NaN,NaN,25.182341,0.544365,1.499343,1.214389,1.452463,1.296200
116,df51eca4-2845-4b35-aa8a-98c40990858b,2023-01-23 14:17:23.000000,karoly@coastlandcivil.com,0.943042,8000.0,285,8000,0.000000,0.000000,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
115,3abf6d7d-9aff-416e-8e4b-ed435e75cbe3,2023-01-23 12:32:46.000000,maasd@saccounty.gov,3.813300,25000.0,284,25000,20.687714,NaN,22.792925,...,NaN,NaN,NaN,NaN,20.687714,0.566750,1.357005,1.377245,1.436384,1.394821
114,bd912081-7e83-4a04-b9ad-5c3f83625114,2023-01-23 12:23:08.000000,dpeterson@co.yuba.ca.us,1.076620,4000.0,283,4000,NaN,NaN,25.838873,...,NaN,NaN,NaN,NaN,25.838873,0.600904,1.385517,1.464811,1.648589,1.449720
113,c814033e-4db8-4989-a568-48fd5109e2fb,2023-01-23 11:50:08.000000,dpeterson@co.yuba.ca.us,0.371074,1300.0,282,1300,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
112,ce1ae6b8-41f9-48d9-8d65-6a56fa9ee39e,2023-01-23 11:48:45.000000,dpeterson@co.yuba.ca.us,0.715406,2700.0,281,2700,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
111,a6982f6a-b7c3-4256-bd09-26a7f145f54b,2023-01-23 11:01:20.000000,justin.lau@cityofwoodland.org,4.688396,7077.0,280,7077,NaN,NaN,17.489264,...,NaN,NaN,NaN,NaN,17.489264,0.670055,1.302474,1.256235,1.242592,1.257183
110,1d0f5fe8-d2ba-49ad-a8a1-9bf1dc079504,2023-01-23 09:58:11.000000,maasd@saccounty.gov,13.105370,30.0,279,30,25.128637,27.823669,NaN,...,NaN,NaN,NaN,NaN,25.128637,0.546905,1.501105,1.220368,1.340317,1.299904


In [42]:
df_jn.loc[df_jn.crash_100mvmt > 100].head()

,project_uid,time_created,user_email,len_mi,aadt,crash_cnt,crash_100mvmt
21,55160fbd-9617-4c09-a05e-da8819a2b71c,2022-12-06 13:47:37,jhong@sacog.org,3.670994,5.0,136,463089.843750
45,fbc41452-2ed7-44ce-b1d1-3563bad0f73c,2023-01-17 06:06:46,rbissegger@markthomas.com,1.861210,58600.0,36,130.566345
